In [1]:
!pip install python-ffmpeg pyannote.audio python-dotenv --quiet
!pip install git+https://github.com/m-bain/whisperx.git --quiet


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import os
import shutil
import glob
import json
from ffmpeg import FFmpeg
import tqdm.auto as tqdm
from pyannote.audio import Pipeline as PyannotePipeline
from pyannote.audio.pipelines.utils.hook import ProgressHook
import dotenv
import torch
import torchaudio 
import pickle
import whisperx
import gc 

from utils import *

dotenv.load_dotenv()

device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


## Set constants

In [3]:
video_root = "../RuPaul's Drag Race"

## Extract audio from videos

In [4]:
os.makedirs("audio_tracks", exist_ok=True)
video_paths = glob.glob(f"{video_root}/*/*", recursive=True)
for video_path in tqdm.tqdm(video_paths):
    audio_path = os.path.join("audio_tracks", ".".join(os.path.basename(video_path).split(".")[:-1]) + ".mp3")
    if os.path.exists(audio_path):
        continue
    FFmpeg().input(video_path).output(audio_path).execute()

0it [00:00, ?it/s]

## Diarize

In [6]:
diarization_pipeline = PyannotePipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1", use_auth_token=os.getenv("HUGGINGFACE_TOKEN")
).to(device)

os.makedirs("diarizations", exist_ok=True)
audio_paths = glob.glob("audio_tracks/*.mp3")
for audio_path in tqdm.tqdm(audio_paths):
    diarization_path = os.path.join("diarizations", ".".join(os.path.basename(audio_path).split(".")[:-1]) + ".pkl")
    if os.path.exists(diarization_path):
        continue
    with ProgressHook() as hook:
        waveform, sample_rate = torchaudio.load(audio_path)
        diarization = diarization_pipeline({"waveform": waveform, "sample_rate": sample_rate}, hook=hook)
        with open(diarization_path, "wb") as f:
            pickle.dump(diarization, f)

  0%|          | 0/8 [00:00<?, ?it/s]